<a href="https://colab.research.google.com/github/onsoon829/dataproject/blob/master/654_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EB%AC%B8%EC%9E%A5_%EA%B0%90%EC%A0%95%EB%B6%84%EB%A5%98_CNN%EB%AA%A8%EB%8D%B8(%EC%88%98%EC%A0%95).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd  '/content/drive/MyDrive/ai_chat_python'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ai_chat_python


# * 문장 영화리뷰 분류 CNN 모델*

In [ ]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import preprocessing
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
import pickle

# 데이터 읽어오기
train_file = "./datasets/chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

print(data.head())
print(data['label'].value_counts()) # 0일상다반사, 1:이별(부정), 2:사랑(긍정)

# 단어 인덱스 시퀀스 벡터: 질문 리스트(features)에서 한 문장씩 꺼내와 text_to_word_sequesce()함수를 이용한다.
# 만든 후 corpus변수에 저장한다.
# 단어 인덱스 스퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]

from collections import OrderedDict

# 텍스트 토큰화시키기
# 가장 빈도수가 높은 1000개의 단으ㅓ만 선택하도록 하는 Tokenizer 객체
tokenizer = preprocessing.text.Tokenizer(num_words=1000)
print(dir(tokenizer))
tokenizer.fit_on_texts(corpus)
print('===================')
print(tokenizer)
print(dir(tokenizer))

with open('vocab.pickle', 'wb') as fp:
  pickle.dump(tokenizer, fp)


# 문장 내 모든 단어를 시퀀스 번호로 변환한다.
sequences = tokenizer.texts_to_sequences(corpus)
print('sequences:' , sequences[0:5])
#print(sequences)
word_index = tokenizer.word_index
#print(word_index)

MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기  pad: padding        시퀀스를 맥스로 해서 15개 채운다
# 단어 시퀀스 벡터 크기, padding = 'pre'(0을 앞에 채움), padding='post' (0을 뒤에 채움)
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')
print(padded_seqs)
print(padded_seqs[0]) # ?

# 학습용, 검증용, 테스트용 데이터셋 생성.
# 학습용, 검증용, 테스트용 = 7:2:1 요 비율대로 나눠준다.
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

train_size = int(len(padded_seqs) *0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)
# 20씩 처리한다는 얘기.
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size+val_size).take(test_size).batch(20)

# 하이퍼파라미터
dropout_prob=0.5
EMB_SIZE=128
EPOCH=5
VOCAB_SIZE = len(word_index) + 1 # 전체 단어 수


# CNN모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN))
print('input_layer :', input_layer.shape)
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
print('embedding_layer :', embedding_layer.shape)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)
print('dropout_emb :', dropout_emb.shape)

# 풀링 연산이란 합성곱 연산 결과로 나온 특징맵의 크기를 줄이거나 주요한 특징을 추출하기 위해서 사용되는 연산이다.
# 풀링 연산에는 최대풀링과 평균풀링 연산이 있는데 주로 최대 풀링 연산을 사용한다.
# 풀링 연산에도 합성곱연산에서 사용되는 윈도우 크기, 스트라이드, 패딩 개념등이 동일하게 사용된다.
# 아래 코드들: 특징 추출
conv1 = Conv1D(filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu)(dropout_emb)
print('conv1 :', conv1.shape)  # conv1: (None, 13, 128)
pool1 = GlobalMaxPool1D()(conv1)
print('pool1 :', pool1.shape)  # pool1: (None, 128)
conv2 = Conv1D(filters=128, kernel_size=4, padding='valid', activation=tf.nn.relu)(dropout_emb)
print('conv2 :', conv2.shape)  # conv2: (None, 12, 128)
pool2 = GlobalMaxPool1D()(conv2)
print('pool2 :', pool2.shape)  # pool2: (None, 128)
conv3 = Conv1D(filters=128, kernel_size=5, padding='valid', activation=tf.nn.relu)(dropout_emb)
print('conv3 :', conv3.shape)  # conv3: (None, 11, 128)
pool3 = GlobalMaxPool1D()(conv3)
print('pool3 :', pool2.shape)  # pool3: (None, 128)

# 3, 4, 5-gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden)
predictions = Dense(3, activation=tf.nn.softmax)(logits)

# 모델생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # 경사하강법 어떻게 적용할지 고민

# 모델학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH)

# 모델평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss:%f' %(loss))

# 모델 저장
model.save('./source/cnn_model.h5')

                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0
0    5290
1    3570
2    2963
Name: label, dtype: int64
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_keras_api_names', '_keras_api_names_v1', 'analyzer', 'char_level', 'document_count', 'filters', 'fit_on_sequences', 'fit_on_texts', 'get_config', 'index_docs', 'index_word', 'lower', 'num_words', 'oov_token', 'sequences_to_matrix', 'sequences_to_texts', 'sequences_to_texts_generator', 'split', 'texts_to_matrix', 'texts_to_sequences', 'texts_to_

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# * 문답 데이터 감정분류 테스트 *

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras.models import Model, load_model
from keras import preprocessing
import pickle

# 데이터 읽어오기
train_file = "./datasets/chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

# 단어 인덱스 시퀀스 벡터 워드로 인덱스 생성해야 한다.
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
#tokenizer = preprocessing.text.Tokenizer()
#tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index

# train에서 파일에 저장해놓은 vocab을 읽어온다.
with open('vocab.pickle', 'rb') as fp:
  tokenizer = pickle.load(fp)


sequences = tokenizer.texts_to_sequences(corpus)
MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20) # 테스트 데이터셋
# 감정 분류 CNN 모델 불러오기
model = load_model('./datasets/cnn_model.h5') # 있는 파일도 없다고 뻥치면 어떡하니ㄷ?
model.summary()
model.evaluate(test_ds, verbose=2)

# 테스트용 데이터셋의 10212번째 데이터 출력
print("단어 시퀀스 : ", corpus[10212])
print("단어 인덱스 시퀀스 : ", padded_seqs[10212])
print("문장 분류(정답) : ", labels[10212])

# 테스트용 데이터셋의 10212번째 데이터 감정 예측
picks = [10212]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print("감정 예측 점수 : ", predict)
print("감정 예측 클래스 : ", predict_class.numpy())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 15)]                 0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 15, 128)              1715072   ['input_2[0][0]']             
                                                                                                  
 dropout_2 (Dropout)         (None, 15, 128)              0         ['embedding_1[0][0]']         
                                                                                                  
 conv1d_3 (Conv1D)           (None, 13, 128)              49280     ['dropout_2[0][0]']           
                                                                                            

단어 시퀀스 :  ['썸', '타는', '여자가', '남사친', '만나러', '간다는데', '뭐라', '해']
단어 인덱스 시퀀스 :  [ 13  61 127 856  31   0   0   0   0   0   0   0   0   0   0]
문장 분류(정답) :  2
1/1 [==============================] - 0s 116ms/step
감정 예측 점수 :  [[1.0889631e-04 3.6781712e-04 9.9952328e-01]]
감정 예측 클래스 :  [2]
